# Домашнее задание Этика искусственного интеллекта

Выполнил: Груданов Николай Алексеевич

---

## Подготовка среды

Подготовим среду для выполнения задания. Загрузим все необходимые библиотеки

In [1]:
!python --version

Python 3.12.7


In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


print(f' pandas version: {pd.__version__}')
print(f' numpy version: {np.__version__}')
print(f' sklearn version: {sklearn.__version__}')

 pandas version: 2.2.2
 numpy version: 1.26.4
 sklearn version: 1.5.1


---

## Подготовка данных

In [3]:
# Загрузим датасет
df = pd.read_csv("data.csv")

In [4]:
# Посмотрим первые 5 строк датафрейма 
display(df.head())

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0000,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
1,239607,0.912500,Yet call out all Muslims for the acts of a few...,0.050000,0.237500,0.612500,0.887500,0.1125,0.0,0.0,...,26670,approved,0,0,0,1,0,0.000000,4,80
2,239612,0.830769,This bitch is nuts. Who would read a book by a...,0.107692,0.661538,0.338462,0.830769,0.0000,0.0,0.0,...,26674,rejected,0,0,0,0,0,0.061538,4,65
3,240311,0.968750,You're an idiot.,0.031250,0.062500,0.000000,0.968750,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.000000,0,32
4,240329,0.900000,Who cares!? Stark trek and Star Wars fans are ...,0.100000,0.200000,0.000000,0.900000,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.300000,0,10


In [5]:
# Посмотрим на статистику датафрейма
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90902 entries, 0 to 90901
Data columns (total 45 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   90902 non-null  int64  
 1   target                               90902 non-null  float64
 2   comment_text                         90902 non-null  object 
 3   severe_toxicity                      90902 non-null  float64
 4   obscene                              90902 non-null  float64
 5   identity_attack                      90902 non-null  float64
 6   insult                               90902 non-null  float64
 7   threat                               90902 non-null  float64
 8   asian                                21687 non-null  float64
 9   atheist                              21687 non-null  float64
 10  bisexual                             21687 non-null  float64
 11  black                       

In [6]:
# Посмотрим на количество пропущенных значений в датафрейме
df.isnull().sum()

id                                         0
target                                     0
comment_text                               0
severe_toxicity                            0
obscene                                    0
identity_attack                            0
insult                                     0
threat                                     0
asian                                  69215
atheist                                69215
bisexual                               69215
black                                  69215
buddhist                               69215
christian                              69215
female                                 69215
heterosexual                           69215
hindu                                  69215
homosexual_gay_or_lesbian              69215
intellectual_or_learning_disability    69215
jewish                                 69215
latino                                 69215
male                                   69215
muslim    

После предварительного просмотра данных выполнил код из условия задания

In [7]:
np.random.seed(0)
comments = df["comment_text"]
target = (df["target"]>0.7).astype(int)

Теперь посмотрим какие данные в наших переменных получились

In [8]:
comments.info()

<class 'pandas.core.series.Series'>
RangeIndex: 90902 entries, 0 to 90901
Series name: comment_text
Non-Null Count  Dtype 
--------------  ----- 
90902 non-null  object
dtypes: object(1)
memory usage: 710.3+ KB


In [9]:
comments.shape

(90902,)

In [10]:
# Посмотрим первые 5 строк
display(comments.head())

0                 haha you guys are a bunch of losers.
1    Yet call out all Muslims for the acts of a few...
2    This bitch is nuts. Who would read a book by a...
3                                     You're an idiot.
4    Who cares!? Stark trek and Star Wars fans are ...
Name: comment_text, dtype: object

In [11]:
target.info()

<class 'pandas.core.series.Series'>
RangeIndex: 90902 entries, 0 to 90901
Series name: target
Non-Null Count  Dtype
--------------  -----
90902 non-null  int64
dtypes: int64(1)
memory usage: 710.3 KB


In [12]:
target.shape

(90902,)

In [13]:
# Посмотрим первые 5 строк
display(target.head())

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

In [14]:
display(target.value_counts())

target
1    45451
0    45451
Name: count, dtype: int64

Перейдем к выполнению заданий

---

## Задания

### Задание 1

Теперь разделим наши данные на train и test. 

Пусть в тест у нас пойдет 30% данных. 

Для этого можете использовать библиотеку train_test_split из sklearn.

In [15]:
# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    comments,  # данные
    target,     # метки
    test_size=0.3,  # 30% — тестовая выборка, 70% — обучающая
    random_state=42  # фиксируем случайность для повторяемости
)

# Выводим результат
print("ТЕСТОВАЯ ВЫБОРКА\n")
print("---------------------------------------")
print(f"Размер тестовой выборки: {X_test.shape}")
print("---------------------------------------")
print(f"Примеры признаков:\n{X_test.head()}")
print("---------------------------------------")
print(f"Примеры тестовых меток:\n{y_test.head()}")

print("\n=======================================\n")

print("ТРЕНИРОВОЧНАЯ ВЫБОРКА\n")
print("---------------------------------------")
print(f"Размер тренировочной выборки: {X_train.shape}")
print("---------------------------------------")
print(f"Примеры тренировочных признаков:\n{X_train.head()}")
print("---------------------------------------")
print(f"Примеры тренировочных меток:\n{y_train.head()}")
print("---------------------------------------")


ТЕСТОВАЯ ВЫБОРКА

---------------------------------------
Размер тестовой выборки: (27271,)
---------------------------------------
Примеры признаков:
80470    Not meaning to belittle your practical concern...
28773    Did you mean to imply that the bears are dying...
76685    As a 9-year dispatch veteran, let me break it ...
12580    A your wimp is traveling across Canada instead...
16111    Oh Dispatch, you right wing media elitist rag....
Name: comment_text, dtype: object
---------------------------------------
Примеры тестовых меток:
80470    0
28773    1
76685    0
12580    1
16111    1
Name: target, dtype: int64


ТРЕНИРОВОЧНАЯ ВЫБОРКА

---------------------------------------
Размер тренировочной выборки: (63631,)
---------------------------------------
Примеры тренировочных признаков:
12294                                    Muslim terrorist.
57506    It's ironic that these are the same groups tha...
56118    Star Wars has a wow factor that Star Trek does...
28624    The settlem

### Задание 2

Преобразуйте текст, который вы поделили на train и test, в числовой формат с помощью этой функции.

In [16]:
# Инициализация векторизатора
vectorizer = CountVectorizer(max_features=200)

# Преобразование датафрема X_train в новый датафрейм, в котором тексты заменены на вектора  
X_train_vec = vectorizer.fit_transform(X_train)


# Преобразование датафрема X_train в новый датафрейм, в котором тексты заменены на вектора  
X_test_vec = vectorizer.fit_transform(X_test)


In [17]:
# Сверяем размерность полученных датасетов 

print("РАЗМЕРНОСТИ ДАТАСЕТОВ\n")
print("---------------------------------------")
print(f"Размерность датасета X_train_vec: {X_train_vec.shape}")
print(f"Размерность датасета y_train: {y_train.shape}")
print("---------------------------------------")

print("\n=======================================\n")

print("---------------------------------------")
print(f"Размерность датасета X_test_vec: {X_test_vec.shape}")
print(f"Размерность датасета y_test: {y_test.shape}")
print("---------------------------------------")

РАЗМЕРНОСТИ ДАТАСЕТОВ

---------------------------------------
Размерность датасета X_train_vec: (63631, 200)
Размерность датасета y_train: (63631,)
---------------------------------------


---------------------------------------
Размерность датасета X_test_vec: (27271, 200)
Размерность датасета y_test: (27271,)
---------------------------------------


### Задание 3

Импортируйте из библиотеки sklearn логистическую регрессию LogisticRegression с параметром max_iter=2000. Для оценки модели возьмите метрику accuracy и посчитайте ее.

In [18]:
# Инициализация и обучение модели логистической регрессии
model = LogisticRegression(max_iter=2000, warm_start=True)
model.fit(X_train_vec, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test_vec)


In [19]:
# Расчет accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f'Метрика Accuracy: {accuracy:.3f}')


Метрика Accuracy: 0.607


### Задание 4

Чтобы мы смогли протестировать разные комментарии, которые приходят в голову, пропишите ниже функцию, для которой на вход мы бы подавали наш комментарий, а на выход получали предсказание, насколько от 0 до 1 комментарий является токсичным.

In [57]:
def predict_toxicity(comment: str)-> None:
   """
   Функция для предсказания токсичности комментария.

   Аргументы:
   comment (str): Комментарий, для которого нужно предсказать токсичность.

   Возвращает:
   None: Функция выводит результат предсказания на экран.
   """
   # Векторизация комментария  
   comment_vec = vectorizer.transform([comment])
   
   # Предсказание
   toxicity_probability = model.predict_proba(comment_vec)[0][1]

   # Вывод результата
   if toxicity_probability > 0.5:
      print("Комментарий токсичный.")
   else:
      print("Комментарий не токсичный.")

   print(f"Вероятность токсичности: {toxicity_probability:.3f}")


### Задание 5

Попробуйте предсказать, токсичен ли комментарий «Apples are stupid». Потом предскажите, токсичен ли комментарий «I love apples».

In [27]:
test_comment_1 = "Apples are stupid"
test_comment_2 = "I love apples"

In [58]:
predict_toxicity(test_comment_1)

Комментарий токсичный.
Вероятность токсичности: 0.509


In [59]:
predict_toxicity(test_comment_2)

Комментарий не токсичный.
Вероятность токсичности: 0.480


### Задание 6

Выведите десять слов, которые считаются наиболее токсичными, а также их коэффициенты.

In [50]:
# Создаем словарь, где ключ - индекс слова, значение - само слово
word_to_index = {word: index for index, word in vectorizer.vocabulary_.items()}

# Получаем коэффициенты модели
model_coefficients = model.coef_[0]

# Получаем индексы 10 самых токсичных слов
top_toxic_indices = np.argsort(model_coefficients)[-10:][::-1]

# Получаем 10 самых токсичных слов и их коэффициенты
top_toxic_words = [(word_to_index[i], model_coefficients[i]) for i in top_toxic_indices]

# Выводим результат
print("10 самых токсичных слов:")
print("---------------------------------------------")
print("№ | Слово | Коэффициент")
print("---------------------------------------------")
# Ипользуем i для нумерации топа слов
i = 1
for word, weight in top_toxic_words:
    print(f"№{i} Слово: {word}, Коэффициент: {weight:.3f}\n")
    i += 1


print("---------------------------------------------")


10 самых токсичных слов:
---------------------------------------------
№ | Слово | Коэффициент
---------------------------------------------
№1 Слово: idiot, Коэффициент: 5.732

№2 Слово: idiots, Коэффициент: 5.659

№3 Слово: such, Коэффициент: 5.510

№4 Слово: ignorant, Коэффициент: 3.741

№5 Слово: who, Коэффициент: 0.946

№6 Слово: man, Коэффициент: 0.563

№7 Слово: they, Коэффициент: 0.419

№8 Слово: two, Коэффициент: 0.389

№9 Слово: left, Коэффициент: 0.318

№10 Слово: work, Коэффициент: 0.297

---------------------------------------------


### Задание 7

Есть ли слова, которых, кажется, не должно быть в списке?

Проанализировав список можно заметить, что часть слов неоднозначена. Эти слова могут быть токсичным, если их использовать для оскорбления или унижения, но они также может быть использовано в нейтральном контексте. Сами по себя слова не оскорбительные и не носят только негативный характер в отличе от слов в топе таких как: idiot, idiots и ignorant, так как они часто используются для оскорбления и унижения других людей. 

Список слов которые под вопросов в этом списке:

- Who - кто

- Man - мужчина. 

- They - они. 

- Two - два. 

- Left - слева, осталось.

- Work - работа.

### Задание 8

Давайте попробуем протестировать модель на ее предвзятость, например, к религии.

In [54]:
test_comment_list = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend",
]

In [60]:
for comment in test_comment_list:
    print(f"Комментарий: {comment}")
    predict_toxicity(comment)
    print("---------------------------------------------")

Комментарий: I have a christian friend
Комментарий не токсичный.
Вероятность токсичности: 0.465
---------------------------------------------
Комментарий: I have a muslim friend
Комментарий не токсичный.
Вероятность токсичности: 0.465
---------------------------------------------
Комментарий: I have a white friend
Комментарий не токсичный.
Вероятность токсичности: 0.454
---------------------------------------------
Комментарий: I have a black friend
Комментарий не токсичный.
Вероятность токсичности: 0.465
---------------------------------------------


Все комментарии попали в группу не токсичных. Под вопросом "I have a black friend", звучит не совсем политкгректно. 

### Задание 9

Вы заметили, что комментарии, относящиеся к исламу, с большей вероятностью будут токсичными, чем комментарии, относящиеся к другим религиям, поскольку онлайн-сообщество исламофобно. Какой тип предвзятости это может внести в вашу модель?

Label bias / Measurement bias - если разметчики или автоматические системы чаще помечают нейтральные сообщения о мусульманах как токсичные из-за своих предубеждений, то модель будет повторять эту ошибку

Модель будет несправедливо завышать вероятность токсичности для комментариев, связанных с исламом, даже если они нейтральныьны или позитивны, и занижать вероятность токсичности для комментариев, связанных с другими религиями, даже если они токсичны.

### Задание 10

Подумайте о том, как можно улучшить алгоритм, чтобы сделать его более этичным. Напишите 1–2 идеи.

1. Эмбеддинги не только отдельных слов, но и словосочетаний (например, биграмм или триграмм), позволяют улавливать более широкий контекст, чем анализ отдельных токенов. Увеличитчение размера окна контекста может помочь модели лучше понимать смысл фраз и выражений, что снижает вероятность ошибочной классификации.

2. Семантические триплеты (например, субъект-глагол-объект) представляют собой структурированный способ извлечения смысла из текста, что может помочь в понимании намерений автора комментария. Это может быть полезно для более точной классификации комментариев за счет более глубокого анализа контекста.

### Задание 11

Бонусное задание (не оценивается)
Придумайте алгоритм, который сможет этично классифицировать комментарии на токсичные и нетоксичные.

В целом описал в 10 задании. Можно добавить использование специальных моделей которые лучши работают с контестом такие как BERT и различные его вариции. 
Также многие коментаторы пытаются обойти фильтры, поэтому можно добавить проверку на написание токсичных коментариев латиницей или с использованием цифр и символов.